In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Huntingtons_Disease"
cohort = "GSE34201"

# Input paths
in_trait_dir = "../../input/GEO/Huntingtons_Disease"
in_cohort_dir = "../../input/GEO/Huntingtons_Disease/GSE34201"

# Output paths
out_data_file = "../../output/preprocess/Huntingtons_Disease/GSE34201.csv"
out_gene_data_file = "../../output/preprocess/Huntingtons_Disease/gene_data/GSE34201.csv"
out_clinical_data_file = "../../output/preprocess/Huntingtons_Disease/clinical_data/GSE34201.csv"
json_path = "../../output/preprocess/Huntingtons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Early transcriptional changes linked to naturally occurring Huntington's disease mutations in human embryonic stem cells"
!Series_summary	"Multiple human embryonic stem (ES) cell lines derived from blastocysts diagnosed as carrying the mutant huntingtin gene by pre-implantation diagnosis were used to explore early developmental changes in gene expression. How mutant huntingtin impacts on signalling pathways in the pre-symptomatic period has remained essentially unexplored in humans due to a previous lack of appropriate models."
!Series_overall_design	"Total RNA was isolated from 10 human ES cell lines, 6 HD and 4 wild type control, and their neural stem cell (NSC) progeny."
Sample Characteristics Dictionary:
{0: ['cell line: VUB01', 'cell line: H9', 'cell line: SA01', 'cell line: SI-187', 'cell line: VUB05', 'cell line: Huez2.3', 'cell line: WT4', 'cell line: SIVF017', 'cell line: SIVF018', 'cell line: SIVF020'], 1: ['hd genotype: wild type', 'hd 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Review the background information and sample characteristics to determine data availability

# 1. Gene Expression Data Availability
# From the background info, this appears to be gene expression data from human ES cell lines
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (HD Status) is available in row 1 of the sample characteristics dictionary
trait_row = 1
# Age is not available in the sample characteristics
age_row = None
# Gender is available in row 3
gender_row = 3

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert HD genotype status to binary value."""
    if not value or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'wild type' in value or 'wildtype' in value or 'wt' in value or 'control' in value:
        return 0
    elif 'hd' in value or 'huntington' in value or 'mutant' in value:
        return 1
    else:
        return None

def convert_gender(value):
    """Convert gender to binary value (0 for female, 1 for male)."""
    if not value or pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.strip().lower()
    
    if 'female' in value or 'f' in value:
        return 0
    elif 'male' in value or 'm' in value:
        return 1
    else:
        return None

# Since age is not available, we don't need a convert_age function
convert_age = None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Save initial filtering information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data as CSV
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM844017': [0.0, 1.0], 'GSM844018': [0.0, 1.0], 'GSM844019': [0.0, 1.0], 'GSM844020': [0.0, 1.0], 'GSM844021': [0.0, 0.0], 'GSM844022': [0.0, 0.0], 'GSM844023': [0.0, 0.0], 'GSM844024': [0.0, 0.0], 'GSM844025': [0.0, 1.0], 'GSM844026': [0.0, 1.0], 'GSM844027': [0.0, 1.0], 'GSM844028': [0.0, 1.0], 'GSM844029': [1.0, 1.0], 'GSM844030': [1.0, 1.0], 'GSM844031': [1.0, 1.0], 'GSM844032': [1.0, 1.0], 'GSM844033': [1.0, 1.0], 'GSM844034': [1.0, 1.0], 'GSM844035': [1.0, 1.0], 'GSM844036': [1.0, 1.0], 'GSM844037': [1.0, 0.0], 'GSM844038': [1.0, 0.0], 'GSM844039': [1.0, 0.0], 'GSM844040': [1.0, 0.0], 'GSM844041': [0.0, 1.0], 'GSM844042': [0.0, 1.0], 'GSM844043': [0.0, 1.0], 'GSM844044': [1.0, 1.0], 'GSM844045': [1.0, 1.0], 'GSM844046': [1.0, 1.0], 'GSM844047': [1.0, 1.0], 'GSM844048': [1.0, 1.0], 'GSM844049': [1.0, 1.0], 'GSM844050': [1.0, 0.0], 'GSM844051': [1.0, 0.0], 'GSM844052': [1.0, 0.0], 'GSM844053': [0.0, 1.0], 'GSM844054': [0.0, 1.0], 'GSM844055': [0.0, 1.0]

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Huntingtons_Disease/GSE34201/GSE34201_series_matrix.txt.gz


Gene data shape: (48803, 72)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, I can see they are ILMN_* format identifiers,
# which are Illumina probe IDs, not standard human gene symbols.
# These need to be mapped to official gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'Species', 'Source', 'Search_Key', 'Transcript', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Unigene_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'GB_ACC']
{'ID': ['ILMN_1825594', 'ILMN_1810803', 'ILMN_1722532', 'ILMN_1884413', 'ILMN_1906034'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['Unigene', 'RefSeq', 'RefSeq', 'Unigene', 'Unigene'], 'Search_Key': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'Transcript': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'ILMN_Gene': ['HS.388528', 'LOC441782', 'JMJD1A', 'HS.580150', 'HS.540210'], 'Source_Reference_ID': ['Hs.3

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping
# From examining the gene annotation data, we can see:
# - 'ID' column contains the Illumina probe IDs (e.g., ILMN_1825594)
# - 'Symbol' column contains the gene symbols (e.g., JMJD1A)

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(preview_df(gene_mapping, n=5))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data after mapping - shape: {gene_data.shape}")
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Save the gene expression data to a CSV file
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (35966, 2)
Gene mapping preview:
{'ID': ['ILMN_1810803', 'ILMN_1722532', 'ILMN_1708805', 'ILMN_1672526', 'ILMN_2185604'], 'Gene': ['LOC441782', 'JMJD1A', 'NCOA3', 'LOC389834', 'C17orf77']}


Gene expression data after mapping - shape: (18839, 72)
First 10 gene symbols after mapping:
['A1BG', 'A2BP1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Gene expression data saved to ../../output/preprocess/Huntingtons_Disease/gene_data/GSE34201.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI database
print("Normalizing gene symbols...")
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {gene_data.shape}")
print("First 10 normalized gene symbols:")
print(gene_data.index[:10])

# Save the normalized gene data
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 2. Load the previously processed clinical data
print("\nLoading clinical data...")
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print("Clinical data preview:")
print(preview_df(clinical_df))

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This GSE34201 dataset contains gene expression data from human embryonic stem cell lines carrying the mutant huntingtin gene, relevant to Huntington's Disease."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: (17552, 72)
First 10 normalized gene symbols:
Index(['A1BG', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS',
       'AACS', 'AACSP1'],
      dtype='object', name='Gene')


Normalized gene data saved to: ../../output/preprocess/Huntingtons_Disease/gene_data/GSE34201.csv

Loading clinical data...
Clinical data preview:
{'GSM844017': [0.0, 1.0], 'GSM844018': [0.0, 1.0], 'GSM844019': [0.0, 1.0], 'GSM844020': [0.0, 1.0], 'GSM844021': [0.0, 0.0], 'GSM844022': [0.0, 0.0], 'GSM844023': [0.0, 0.0], 'GSM844024': [0.0, 0.0], 'GSM844025': [0.0, 1.0], 'GSM844026': [0.0, 1.0], 'GSM844027': [0.0, 1.0], 'GSM844028': [0.0, 1.0], 'GSM844029': [1.0, 1.0], 'GSM844030': [1.0, 1.0], 'GSM844031': [1.0, 1.0], 'GSM844032': [1.0, 1.0], 'GSM844033': [1.0, 1.0], 'GSM844034': [1.0, 1.0], 'GSM844035': [1.0, 1.0], 'GSM844036': [1.0, 1.0], 'GSM844037': [1.0, 0.0], 'GSM844038': [1.0, 0.0], 'GSM844039': [1.0, 0.0], 'GSM844040': [1.0, 0.0], 'GSM844041': [0.0, 1.0], 'GSM844042': [0.0, 1.0], 'GSM844043': [0.0, 1.0], 'GSM844044': [1.0, 1.0], 'GSM844045': [1.0, 1.0], 'GSM844046': [1.0, 1.0], 'GSM844047': [1.0, 1.0], 'GSM844048': [1.0, 1.0], 'GSM844049': [1.0, 1.0], 'GSM844050': [1.0, 0.0], 'G

Linked data shape after handling missing values: (72, 17554)

Checking for bias in dataset features...
For the feature 'Huntingtons_Disease', the least common label is '0.0' with 30 occurrences. This represents 41.67% of the dataset.
The distribution of the feature 'Huntingtons_Disease' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 22 occurrences. This represents 30.56% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Huntingtons_Disease/GSE34201.csv
